## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-11-03-40-00 +0000,wsj,Person Detained for Questioning in Connection ...,https://www.wsj.com/us-news/images-of-masked-p...
1,2026-02-11-03-37-00 +0000,wsj,"Nine Killed, at Least 25 Injured in Canadian S...",https://www.wsj.com/world/americas/nine-killed...
2,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
3,2026-02-11-03-35-59 +0000,nyt,10 Dead in Shootings at School and Home in Bri...,https://www.nytimes.com/2026/02/10/world/canad...
4,2026-02-11-03-34-22 +0000,nyt,House Defeats Republican Bid to Block Votes on...,https://www.nytimes.com/2026/02/10/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2438/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,44
644,epstein,20
141,new,19
5,guthrie,13
776,market,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,146
178,2026-02-10-20-05-05 +0000,bbc,Under-fire Trump commerce secretary confirms h...,https://www.bbc.com/news/articles/cwyx9vq0ne0o...,95
159,2026-02-10-20-53-18 +0000,nyt,Former Palm Beach Police Chief Said Trump Told...,https://www.nytimes.com/2026/02/10/us/politics...,95
162,2026-02-10-20-48-27 +0000,bbc,Ex-police chief said Trump told him in 2006 'e...,https://www.bbc.com/news/articles/cx2407jrn83o...,92
46,2026-02-11-01-14-19 +0000,latimes,L.A. County officials push new sales tax to of...,https://www.latimes.com/california/story/2026-...,91


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,146,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
150,74,2026-02-10-21-23-59 +0000,wapo,"New photos, video show armed person appearing ...",https://www.washingtonpost.com/nation/2026/02/...
204,52,2026-02-10-19-07-52 +0000,nypost,Howard Lutnick admits to senators he visited E...,https://nypost.com/2026/02/10/us-news/lutnick-...
258,47,2026-02-10-16-15-01 +0000,nypost,Gavin Newsom ripped by LA’s top fed prosecuto...,https://nypost.com/2026/02/10/us-news/gavin-ne...
261,41,2026-02-10-16-07-54 +0000,nypost,Nervous Netanyahu pushes up trip to DC for hig...,https://nypost.com/2026/02/10/us-news/nervous-...
100,37,2026-02-10-23-03-05 +0000,latimes,$600 million in Trump administration health cu...,https://www.latimes.com/politics/story/2026-02...
155,37,2026-02-10-21-15-41 +0000,nypost,LI parents who tragically lost hockey star son...,https://nypost.com/2026/02/10/us-news/li-paren...
63,35,2026-02-11-00-12-21 +0000,latimes,LAPD to train their body cameras on immigratio...,https://www.latimes.com/california/story/2026-...
274,33,2026-02-10-15-16-53 +0000,nypost,"Ex-Google CEO Eric Schmidt, 70, spotted with 2...",https://nypost.com/2026/02/10/business/ex-goog...
344,33,2026-02-10-08-46-21 +0000,wsj,"Stock Market News, Feb. 10, 2026: Financial St...",https://www.wsj.com/livecoverage/stock-market-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
